Initial idea is to verify that press releases affect the stock price. 

1. Test that press releases affect the price
2. Get the press release of that stock for that day
    a. Do a sentiment analysis on the title
    b. Check if + sentiment sent the stock up or vice versa


Data gotten here: https://www.biopharmcatalyst.com/calendars/historical-catalyst-calendar

In [1]:
import pandas as pd
import json

pd.set_option("display.max_columns", None)

In [2]:
# Load JSON from a file
with open("historical-catalyst-calendar.json", "r") as file:
    data = json.load(file)

data = data["data"]["data"]

In [3]:
clean_data = []

bad_keys = ["price_change_sparkline", "statuses"]

for item in data:
    for key in bad_keys:
        if key in item:
            del item[key]
    clean_data.append(item)

In [4]:
df = pd.DataFrame(clean_data)
good_cols = ["date", "note", "company_ticker"]
dfc = df[good_cols]

In [6]:
df.to_csv("historical-catalyst-calendar.csv", index=False)